In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

fred_api_key = os.getenv('fred_api_key')


In [2]:
from envoyer import Envoyer
from data_sources.fred import FredDataSource
from db import DuckDBUtility


## possiblly inject custom sql statements to create tables for each data source.
duckdb_util = DuckDBUtility(uri='FRED.duckdb', db_name='timeseries_data')

envoy = Envoyer(db_utility=duckdb_util, db_name="FRED")

fred_data_source = FredDataSource(source_id='fred', api_key=fred_api_key)

envoy.register_data_source("fred", fred_data_source)

# Fetch and save data to database
data = envoy.fetch_data("fred", save_to_db=True, collection_name="fred_data_collection", series_id="GDP").to_format(format="pandas")

In [11]:

#this needs to be implemented from fetch_data method. To do so we need to add a start and end date, then we can use the following code to fetch data then check if we are missing any data, then fetch the missing data and save it to the database.
import duckdb

duckdb_con = duckdb.connect(f'FRED.duckdb')
df = duckdb_con.sql(f"SELECT * FROM timeseries_data").df()
duckdb_con.close()

df

,timestamp,source_id,series_id,value,notes,year,month,day
0,1947-01-01,fred,GDP,243.164,FRED observation,1947,1,1
1,1947-04-01,fred,GDP,245.968,FRED observation,1947,4,1
2,1947-07-01,fred,GDP,249.585,FRED observation,1947,7,1
3,1947-10-01,fred,GDP,259.745,FRED observation,1947,10,1
4,1948-01-01,fred,GDP,265.742,FRED observation,1948,1,1
...,...,...,...,...,...,...,...,...
303,2022-10-01,fred,GDP,26408.405,FRED observation,2022,10,1
304,2023-01-01,fred,GDP,26813.601,FRED observation,2023,1,1
305,2023-04-01,fred,GDP,27063.012,FRED observation,2023,4,1
306,2023-07-01,fred,GDP,27610.128,FRED observation,2023,7,1


In [7]:
from envoyer import Envoyer
from data_sources.fred import FredDataSource
from db import DuckDBUtility

envoy = Envoyer()

fred_data_source = FredDataSource(source_id='fred', api_key=fred_api_key)
envoy.register_data_source("fred", fred_data_source)

# Fetch and and send to dataframes
data_pandas = envoy.fetch_data("fred", series_id="GDP").to_format(format="pandas")
data_pyarrow = envoy.fetch_data("fred", series_id="GDP").to_format(format="pyarrow")